# Notebook 03 - Perform a deep-learning attack

<i>This material has been designed by Azade Rezaeeade and Lizzy Grootjen. Distribution without permission is not permitted.</i>

You've already done a statistical attack, but can we use artificial intelligence to perform the task for us? AI is known for recognising patterns in large amounts of data. Let's find out!
First, load in the python imports, auxiliary arrays and auxiliary functions.

In [23]:
# Python imports

import tensorflow as tf
import numpy as np
from tensorflow.keras.models import load_model

In [12]:
# Auxiliary arrays

sbox = np.array([99, 124, 119, 123, 242, 107, 111, 197, 48,  1,   103, 43,  254, 215, 171, 118, 
                202, 130, 201, 125, 250, 89,  71,  240, 173, 212, 162, 175, 156, 164, 114, 192, 
                183, 253, 147, 38,  54,  63,  247, 204, 52,  165, 229, 241, 113, 216, 49,  21, 
                4,   199, 35,  195, 24,  150, 5,   154, 7,   18,  128, 226, 235, 39,  178, 117, 
                9,   131, 44,  26,  27,  110, 90,  160, 82,  59,  214, 179, 41,  227, 47,  132, 
                83,  209, 0,   237, 32,  252, 177, 91,  106, 203, 190, 57,  74,  76,  88,  207,
                208, 239, 170, 251, 67,  77,  51,  133, 69,  249, 2,   127, 80,  60,  159, 168, 
                81,  163, 64,  143, 146, 157, 56,  245, 188, 182, 218, 33,  16,  255, 243, 210, 
                205, 12,  19,  236, 95,  151, 68,  23,  196, 167, 126, 61,  100, 93,  25,  115, 
                96,  129, 79,  220, 34,  42,  144, 136, 70,  238, 184, 20,  22,  94,  11,  219, 
                224, 50,  58,  10,  73,  6,   36,  92,  194, 211, 172, 98,  145, 149, 228, 121, 
                231, 200, 55,  109, 141, 213, 78,  169, 108, 86,  244, 234, 101, 122, 174, 8, 
                186, 120, 37,  46,  28,  166, 180, 198, 232, 221, 116, 31,  75,  189, 139, 138, 
                112, 62,  181, 102, 72,  3,   246, 14,  97,  53,  87,  185, 134, 193, 29,  158, 
                225, 248, 152, 17,  105, 217, 142, 148, 155, 30,  135, 233, 206, 85,  40,  223, 
                140, 161, 137, 13,  191, 230, 66,  104, 65,  153, 45,  15,  176, 84,  187, 22]) 


In [ ]:
# Auxiliary functions

# Load in the dataset using numpy
def load_dataset(path):
    dataset = np.load(path)
    traces = dataset["trace"]
    textin = dataset["textin"]
    return traces, textin

# Decode the bytes to text using ASCII
def decode_bytes_to_text(predicted_key):
    password = ""
    for elem in predicted_key:
        elem_char = chr(elem)
        password = password + elem_char
    return password

## Exercise 1: Load in the dataset
By now, you probably aren't surprised that you have to load in your dataset :-)

You get a new set of traces with an unknown password. Let's crack it using AI!

To do:
- Fill in the blank with loading in the traces and plaintexts
- How many traces do you have? And how many samples? 

In [ ]:
## Exercise
path = "../../datasets/dataset_nb03.npy"

traces, plaintexts = ...

traces_shape = ...

num_traces = ...
num_samples = ...

print("Shape of dataset: " + str(traces_shape))
print("Number of traces: " + str(num_traces))
print("Number of samples: " + str(num_samples))

## Exercise 2: recover one byte using AI

We already did the AI (deep neural network models) training for you! In order to crack the whole password (16 bytes), we have a prediction model for each byte. 
The deep neural network models are located in "pretrained-models/models". To get back the predicted keybyte from the deep neural network, we have provided you with an auxiliary function to use. 

Let's try to crack the first byte. We trained our models using tensorflow. Import the first model and try to predict the first passwordbyte with the provided dataset.

To do:
- Import the first model using [tf.keras.model.load()](https://www.tensorflow.org/api_docs/python/tf/keras/models/load_model). 
- Predict the passwordbyte using [model.predict()](https://www.tensorflow.org/api_docs/python/tf/keras/Model#predict). 
- Use the auxiliary function retreive_key() to get the passwordbyte from the predictions.

In [ ]:
# This function saves you the hassle of going back from intermediate values to the password.
# It takes the predictions, plaintexts and the byte you want to attack and returns the predicted passwordbyte.
def retrieve_key(predictions, plaintexts, target_byte):
    key_bytes_proba = np.zeros(256)
    plaintext = plaintexts[:,target_byte]

    for i in range(0, 256): # for every password candidate, check if its matching sbox value has the highest probability
        sbox_proba = predictions[:,sbox[plaintext^i]]
        proba_summed = np.sum(sbox_proba) # sum all probabilities for each timesample
        key_bytes_proba[i] += np.log(proba_summed)

    return np.argmax(key_bytes_proba)

In [ ]:
## Exercise
attack_byte = ...
path_to_first_model = "../../pretrained-models/models/model_byte_{}.keras".format(attack_byte)

model_firstbyte = ...

predictions = ...

predicted_keybyte = ...

print("The predicted key value for byte {} is:  {}".format(attack_byte, predicted_keybyte))
print("This belongs to the character: {}".format(decode_bytes_to_text([predicted_keybyte])))

## Exercise 3: Recover all bytes using AI

If you can do it for one byte, then you can do it for all bytes! Recover the key with the corresponding AI-models. The easiest way is to loop over all pretrained models. We have created a small setup for you.

To do:
- Determine how many bytes we need to attack. What was the size of the key?
- Fill in the blanks of the for-loop.
- What message did you find?

In [ ]:
## Exercise
amount_of_bytes_to_attack = ...
predicted_key = np.zeros(amount_of_bytes_to_attack)

for byte in range(0, ...):
    model_attackbyte = ...
    predictions = ...
    predicted_keybyte = ...
    predicted_key[byte] = predicted_keybyte


print("The predicted key is:  {}".format(predicted_key))
print("Translated to text: {}".format(decode_bytes_to_text(predicted_key)))

# Done!
You've succesfully completed this notebook. Did you find the hidden message?

This kind of attack is called a profiled deep-learning side-channel attack. In a profiled attack, we first create a profile of our target device. This target device is in full control of the attacker. The attacker can control the messages and keys. In this notebook, we have created the profile with the use of deep neural networks. They model the power consumption behaviour of the device, by teaching which power trace belongs to which key value. There are different ways to create such profiles, varying from different AI-techniques to more statistical approaches.